In [1]:
#求query_prediction和title的相似度
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
train_data = pd.read_table('oppo_round1_train_20180929.txt',names=[0,1,2,3,4],header=None,encoding='utf8').astype(str)
train_data = train_data[train_data[4] != '音乐' ]
test_data = pd.read_table('oppo_round1_vali_20180929.txt',header=None) #用验证集


train_data.columns = ['prefix','query_prediction','title','tag','label']
test_data.columns = ['prefix','query_prediction','title','tag','label']

In [2]:
train_data[train_data['tag'] == '推广']
train_data = train_data.drop([114987,244937,860181,920381])

In [3]:
test_data = test_data.drop([12237])

In [12]:
#取的是query中最大概率的prediction与title的相似度
import Levenshtein
import json
train_data['query_title_sm'] = 0
test_data['query_ttile_sm'] = 0
def get_similarity(str1,str2):
    return Levenshtein.ratio(str1,str2)
def query_title_sm(item):
    if item['query_prediction'] == '{}':
        return 0 
    else :
        obj = json.loads(item['query_prediction'])
        return get_similarity(sorted(obj.items(),key = lambda x:x[1],reverse = True)[0][0],item['title'])
train_data['query_title_sm'] = train_data.apply(query_title_sm,axis=1)
test_data['query_title_sm'] = test_data.apply(query_title_sm,axis=1)

In [13]:
query_title_sm1 = train_data['query_title_sm']

In [22]:
query_title_sml = query_title_sm1.reset_index()
query_title_sml = query_title_sml.drop(['index'],axis=1)

In [15]:
query_title_sm2 = test_data['query_title_sm']
query_title_sm2 = query_title_sm2.reset_index()
query_title_sm2 = query_title_sm2.drop(['index'],axis=1)

In [16]:
t = pd.read_csv('train_data_10_4_2.csv')
tt = pd.read_csv('test_data_10_4_2.csv')

In [19]:
t = t.reset_index()
t = t.drop(['index'],axis=1)

tt = tt.reset_index()
tt = tt.drop(['index'],axis=1)

In [24]:
query_title_sm1 = query_title_sm1.drop(['index'],axis=1)

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [161]:
query_title_sm1 = pd.DataFrame(query_title_sm1)

In [163]:
query_title_sm2 = pd.DataFrame(query_title_sm2)

In [164]:
train_data = pd.concat([t,query_title_sm1],axis=1)
test_data = pd.concat([tt,query_title_sm2],axis=1)

In [176]:
train_data.to_csv('train_data_10_4_2.csv',encoding='utf-8',index='False')
test_data.to_csv('test_data_10_4_2.csv',encoding='utf-8',index='False')

In [175]:
# train_X_data = train_X_data.drop(['Unnamed: 0'],axis=1)
# test_X_data = test_X_data.drop(['Unnamed: 0'],axis=1)
train_X_data['totalsim'] = train_X_data['query_title_sm']+train_X_data[0]
train_X_data['totalsim2'] = train_X_data[0]*train_X_data['prefix_len']
train_X_data['prefix_tag_ctr2'] = train_X_data['prefix_tag_ctr']*train_X_data['prefix_len']
test_X_data['totalsim'] = test_X_data['totalsim']*test_X_data[0]
test_X_data['totalsim2'] = test_X_data[0]*test_X_data['prefix_len']
test_X_data['prefix_tag_ctr2'] = test_X_data['prefix_tag_ctr']*test_X_data['prefix_len']
# test_X_data['totalsim'] = test_X_data['query_title_sm']+test_X_data[0]
# train_X_data['totalsim'] = train_X_data['totalsim']*train_X_data['prefix_len']
# test_X_data['totalsim'] = test_X_data['totalsim']*test_X_data['prefix_len']

,prefix_pred_0,prefix_pred_1,title_pred_0,title_pred_1,label,健康,商品,应用,影视,快应用,...,音乐,title_ctr,prefix_len,prefix_ctr,tag_ctr,prefix_tag_ctr,title_tag_ctr,prefix_title_ctr,smi_pre_ti,query_title_sm
0,0.624000,0.376000,0.641026,0.358974,0,0,0,0,0,0,...,0,0.382979,2,0.386667,0.263159,0.065217,0.065217,0.382979,1.000000,0.666667
1,0.627901,0.372099,0.627901,0.372099,0,1,0,0,0,0,...,0,0.000000,4,0.500000,0.297379,0.000000,0.000000,0.000000,0.421053,0.384615
2,0.627901,0.372099,0.627901,0.372099,1,0,0,0,0,0,...,0,1.000000,4,0.500000,0.396154,1.000000,1.000000,1.000000,0.888889,0.500000
3,0.647059,0.352941,0.386503,0.613497,1,0,0,0,0,0,...,0,0.620690,2,0.360915,0.413918,0.634021,0.620690,0.634021,0.400000,0.769231
4,0.627451,0.372549,0.484375,0.515625,0,0,0,0,0,0,...,0,0.493333,4,0.367568,0.396154,0.469697,0.506849,0.469697,0.666667,0.588235


In [35]:
##统计每个搜索词出现的次数
import pickle
totaldata = train_data.append(test_data,ignore_index=True)
kk = totaldata.groupby( ['prefix'] )
countmap = {}
for name , group in kk:
    countmap[name] = len(group)
f = open('e:/countmap.pickle','wb')
pickle.dump( countmap , f )

In [9]:
f.close()

In [36]:
##待会儿记着删了
import pickle
f = open('e:/countmap.pickle','rb')
countmap = pickle.load( f )
def getcount( item ):
    return countmap.get(item,0)
train_data['ocrcount'] = train_data['prefix'].map(  getcount )
test_data['ocrcount'] = test_data['prefix'].map(  getcount )

In [37]:
train_data['ocrcount'].to_csv('d:/count.csv',index=False)
test_data['ocrcount'].to_csv('d:/testcount.csv',index=False)


In [32]:
test_data

,prefix,query_prediction,title,tag,label,query_ttile_sm,query_title_sm,ocrcount
0,重庆旅游,"{""重庆旅游景区"": ""0.018"", ""重庆旅游攻略"": ""0.373"", ""重庆旅游景点...",皇包车旅行,应用,0,0,0.181818,59
1,婆婆来了,"{""婆婆来了大结局"": ""0.021"", ""婆婆来了电视剧"": ""0.100"", ""婆婆来了...",搜狐视频,应用,0,0,0.181818,8
2,张国荣,"{""张国荣遗体很恐怖"": ""0.020"", ""张国荣头像"": ""0.013"", ""张国荣24...",张国荣,歌手,0,0,0.750000,946
3,陌陌,"{""陌陌晒奶"": ""0.008"", ""陌陌下载安装"": ""0.009"", ""陌陌交友"": ""...",陌陌,百科,0,0,0.666667,3760
4,畅游,"{""畅游阁"": ""0.322"", ""畅游六零年代"": ""0.134"", ""畅游书城82212...",李畅游,百科,0,0,0.666667,14
5,速腾,"{""速腾2018款"": ""0.008"", ""速腾1.6"": ""0.023"", ""速腾报价"":...",全新速腾,百科,0,0,0.400000,224
6,学位,"{""学位英语"": ""0.032"", ""学位证"": ""0.045"", ""学位网"": ""0.10...",学位证书编号怎么查,经验,1,0,0.333333,19
7,汽车,"{""汽车报价"": ""0.010"", ""汽车标志大全"": ""0.020"", ""汽车之家"": ""...",汽车之家,应用,0,0,1.000000,20066
8,骏,"{""骏马图片"": ""0.016"", ""骏马奔驰保边疆"": ""0.040"", ""骏派a70"":...",骏派A50,汽车,1,0,0.800000,75
9,支,"{""支付宝下载"": ""0.012"", ""支气管炎"": ""0.010"", ""支付宝提现要手续费...",支付宝官网,网站,1,0,0.750000,3722


In [45]:
#!/usr/bin/python
# coding=utf-8
## 广告点击率平滑
import numpy
import random
import scipy.special as special
import math
from math import log


class HyperParam(object):
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def sample_from_beta(self, alpha, beta, num, imp_upperbound):
        sample = numpy.random.beta(alpha, beta, num)
        I = []
        C = []
        for click_ratio in sample:
#             print(click_ratio)
            imp = random.random() * imp_upperbound
            #imp = imp_upperbound
            click = imp * click_ratio
            I.append(imp)
            C.append(click)
        return I, C

    def update_from_data_by_FPI(self, tries, success, iter_num, epsilon):
        '''estimate alpha, beta using fixed point iteration'''
        for i in range(iter_num):
            new_alpha, new_beta = self.__fixed_point_iteration(tries, success, self.alpha, self.beta)
            if abs(new_alpha-self.alpha)<epsilon and abs(new_beta-self.beta)<epsilon:
                break
            self.alpha = new_alpha
            self.beta = new_beta

    def __fixed_point_iteration(self, tries, success, alpha, beta):
        '''fixed point iteration'''
        sumfenzialpha = 0.0
        sumfenzibeta = 0.0
        sumfenmu = 0.0
        for i in range(len(tries)):
            sumfenzialpha += (special.digamma(success[i]+alpha) - special.digamma(alpha))
            sumfenzibeta += (special.digamma(tries[i]-success[i]+beta) - special.digamma(beta))
            sumfenmu += (special.digamma(tries[i]+alpha+beta) - special.digamma(alpha+beta))

        return alpha*(sumfenzialpha/sumfenmu), beta*(sumfenzibeta/sumfenmu)

    def update_from_data_by_moment(self, tries, success):
        '''estimate alpha, beta using moment estimation'''
        mean, var = self.__compute_moment(tries, success)
        #print 'mean and variance: ', mean, var
        #self.alpha = mean*(mean*(1-mean)/(var+0.000001)-1)
        self.alpha = (mean+0.000001) * ((mean+0.000001) * (1.000001 - mean) / (var+0.000001) - 1)
        #self.beta = (1-mean)*(mean*(1-mean)/(var+0.000001)-1)
        self.beta = (1.000001 - mean) * ((mean+0.000001) * (1.000001 - mean) / (var+0.000001) - 1)

    def __compute_moment(self, tries, success):
        '''moment estimation'''
        ctr_list = []
        var = 0.0
        for i in range(len(tries)):
            ctr_list.append(float(success[i])/tries[i])
        mean = sum(ctr_list)/len(ctr_list)
        for ctr in ctr_list:
            var += pow(ctr-mean, 2)

        return mean, var/(len(ctr_list)-1)



def test():
    hyper = HyperParam(1, 1)
    #--------sample training data--------
    I, C = hyper.sample_from_beta(10, 1000, 10000, 1000)
    print(I, C)

    #--------estimate parameter using fixed-point iteration--------
    hyper.update_from_data_by_FPI(I, C, 1000, 0.00000001)
#     print(hyper.alpha, hyper.beta)

    #--------estimate parameter using moment estzimation--------
    hyper.update_from_data_by_moment(I, C)
    print(hyper.alpha, hyper.beta)

[384.13313291308395, 124.16058632000637, 910.0670766066174, 370.6246571288467, 98.72321076647906, 823.6848016264767, 661.906338107715, 338.281502810884, 418.0653251290515, 543.2407684107839, 36.860784932269965, 547.8247002056248, 40.1665385245843, 784.2256719166264, 16.115361059044876, 715.9654214336358, 50.91890638735852, 549.301364995562, 866.4191124963442, 112.35976144638882, 656.2428807508271, 385.58319691322174, 27.606962898936715, 126.29192584237991, 78.78421760218323, 593.2164090010937, 158.39274284535287, 426.5459318936174, 928.7429650130729, 118.07011844418224, 410.8800167378651, 522.9430397317348, 619.8789546548566, 306.99903833511286, 376.88221118456545, 54.91477749679863, 371.1307703902236, 770.6431949295982, 523.7340257929167, 219.21593608812984, 918.1427672613065, 788.9424792685264, 8.116564577148043, 462.72637452554477, 492.88480186537777, 111.78590574868208, 867.6522579024075, 121.80203464862372, 344.334933680163, 746.6159842415052, 688.8373448326752, 619.7901001762405,

8.989413669121577 894.8331697150919


[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000] [499.5818760910904, 533.7647801003892, 513.7169192782067, 486.7660918296135, 471.105914695495, 507.2083844592182, 473.5907712839382, 503.58676914671344, 477.5745403074871, 483.88657476838773]


In [66]:
#!/usr/bin/python
# coding=utf-8
##这里是用贝叶斯进行点击率平滑
import numpy
import random
import scipy.special as special
 
 
class BayesianSmoothing(object):
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta
 
    def sample(self, alpha, beta, num, imp_upperbound):
        sample = numpy.random.beta(alpha, beta, num)
#         print(sample)
        I = []
        C = []
        for clk_rt in sample:
            imp = imp_upperbound
            clk = imp * clk_rt
            I.append(imp)
            C.append(clk)
#         print(C)
        return I, C
 
    def update(self, imps, clks, iter_num, epsilon):
        for i in range(iter_num):
            new_alpha, new_beta = self.__fixed_point_iteration(imps, clks, self.alpha, self.beta)
            if abs(new_alpha-self.alpha)<epsilon and abs(new_beta-self.beta)<epsilon:
                break
            self.alpha = new_alpha
            self.beta = new_beta
 
    def __fixed_point_iteration(self, imps, clks, alpha, beta):
        numerator_alpha = 0.0
        numerator_beta = 0.0
        denominator = 0.0
 
        for i in range(len(imps)):
            numerator_alpha += (special.digamma(clks[i]+alpha) - special.digamma(alpha))
            numerator_beta += (special.digamma(imps[i]-clks[i]+beta) - special.digamma(beta))
            denominator += (special.digamma(imps[i]+alpha+beta) - special.digamma(alpha+beta))
 
        return alpha*(numerator_alpha/denominator), beta*(numerator_beta/denominator)
 
 
 
def main():
    bs = BayesianSmoothing(1, 1)
    I, C = bs.sample(500, 500, 10, 1000)
    print(I,C)
    bs.update(I, C, 1000, 0.0000000001)
#     print(bs.alpha, bs.beta)
    ctr = []
    for i in range(len(I)):
        ctr.append((C[i]+bs.alpha)/(I[i]+bs.alpha+bs.beta))
#     print(ctr)
 
 
 
if __name__ == '__main__':
    main()

totaldata = train_data.append( test_data , ignore_index=True )
totaldata['realtitle'] = totaldata['title']+totaldata['tag']
kkk = totaldata.groupby(['prefix','realtitle'])
namelist = []
clicklist = []
countlist = []
for name , group in kkk:
    namelist.append(name) 
    clicklist.append( len( group['label'][group['label'] == 1]  ) ) 
    countlist.append(len( group ))

In [68]:
bs = BayesianSmoothing(1, 1)
    print(I,C) 
bs.update(countlist, clicklist, 1000, 0.0000000001)
    print(bs.alpha, bs.beta)
ctrmap = {}
for i in range(len(clicklist)):
    ctrmap[namelist[i]] =  (clicklist[i]+bs.alpha)/(countlist[i]+bs.alpha+bs.beta) 
#     ctr.append(( namelist[i] ,  (clicklist[i]+bs.alpha)/(countlist[i]+bs.alpha+bs.beta) ))
#     print(ctr)

In [80]:
##将平滑后的概率重新注入进去
test_data['realtitle'] = test_data['title']+test_data['tag']
def getxx( x ):
    return ctrmap[( x['prefix'],x['realtitle'] )]
# train_data['ph_pred_ctr'] = train_data.apply( getxx , axis =1 )
test_data['ph_pred_ctr'] = test_data.apply( getxx , axis =1  )
train_data['ph_pred_ctr'].to_csv('d:/predctr.csv',index=False)
test_data['ph_pred_ctr'].to_csv('d:/testpredctr.csv',index=False)

In [81]:
test_data['ph_pred_ctr']

0        0.008309
1        0.009521
2        0.000531
3        0.001688
4        0.008677
5        0.002343
6        0.052996
7        0.001382
8        0.050404
9        0.006059
10       0.010547
11       0.007103
12       0.068083
13       0.010547
14       0.010007
15       0.031008
16       0.038820
17       0.011147
18       0.021521
19       0.006406
20       0.004470
21       0.009079
22       0.061123
23       0.000948
24       0.001382
25       0.046777
26       0.004393
27       0.030224
28       0.010007
29       0.004991
           ...   
49970    0.007659
49971    0.068083
49972    0.068083
49973    0.010547
49974    0.068083
49975    0.011147
49976    0.163354
49977    0.011147
49978    0.068083
49979    0.118285
49980    0.194577
49981    0.010547
49982    0.068083
49983    0.061742
49984    0.068083
49985    0.011147
49986    0.068083
49987    0.011147
49988    0.011147
49989    0.011147
49990    0.064416
49991    0.068083
49992    0.010007
49993    0.068083
49994    0

In [ ]:
#求prefix有多少个tag

train_prefix_group = train_data.groupby(['prefix'])
train_data['prefix_tag_count'] = 0
count = 0
for name , group in train_prefix_group:
    count = count + 1
    train_data[train_data['prefix'] == name]['prefix_tag_count']  =  len(group['tag'].unique())
    print(100*count/len(train_prefix_group))
    
test_prefix_group = test_data.groupby(['prefix'])
test_data['prefix_tag_count'] = 0
count = 0
for name , group in test_prefix_group:
    count = count + 1
    test__data[test_data['prefix'] == name]['prefix_tag_count']  =  len(group['tag'].unique())
    print(100*count/len(test_prefix_group))

0.0006366101781235279
0.0012732203562470558
0.0019098305343705835
0.0025464407124941115
0.0031830508906176393
0.003819661068741167
0.004456271246864695
0.005092881424988223
0.00572949160311175
0.006366101781235279
0.007002711959358806
0.007639322137482334
0.008275932315605862
0.00891254249372939
0.009549152671852917
0.010185762849976446
0.010822373028099973
0.0114589832062235
0.012095593384347028
0.012732203562470557
0.013368813740594085
0.014005423918717612
0.014642034096841141
0.015278644274964668
0.015915254453088196
0.016551864631211725
0.01718847480933525
0.01782508498745878
0.01846169516558231
0.019098305343705834
0.019734915521829363
0.020371525699952892
0.021008135878076418
0.021644746056199947
0.022281356234323476
0.022917966412447
0.02355457659057053
0.024191186768694056
0.024827796946817585
0.025464407124941114
0.02610101730306464
0.02673762748118817
0.027374237659311698
0.028010847837435224
0.028647458015558753
0.029284068193682282
0.029920678371805808
0.030557288549929337


0.26355661374314054
0.26419322392126404
0.2648298340993876
0.2654664442775111
0.26610305445563465
0.26673966463375814
0.2673762748118817
0.2680128849900052
0.26864949516812875
0.2692861053462523
0.2699227155243758
0.27055932570249935
0.27119593588062285
0.2718325460587464
0.2724691562368699
0.27310576641499346
0.27374237659311695
0.2743789867712405
0.275015596949364
0.27565220712748756
0.27628881730561106
0.2769254274837346
0.27756203766185816
0.27819864783998166
0.2788352580181052
0.2794718681962287
0.28010847837435227
0.28074508855247576
0.2813816987305993
0.2820183089087228
0.28265491908684637
0.28329152926496987
0.2839281394430934
0.2845647496212169
0.28520135979934047
0.285837969977464
0.2864745801555875
0.2871111903337111
0.2877478005118346
0.2883844106899581
0.2890210208680816
0.2896576310462052
0.2902942412243287
0.29093085140245223
0.2915674615805757
0.2922040717586993
0.29284068193682283
0.29347729211494633
0.2941139022930699
0.2947505124711934
0.29538712264931694
0.296023732

In [ ]:
#求title有多少个tag\
train_title_group = train_data.groupby(['title'])
train_data['title_tag_count'] = 0
count = 0
for name,group in train_title_group:
    train_data[train_data['title'] == name]['title_tag_count']  =  len(group['tag'].unique())
    print(100*count/len(train_title_group))
    
test_title_group = test_data.groupby(['title'])
test_data['title_tag_count'] = 0
count = 0
for name,group in train_title_group:
    test_data[test_data['title'] == name]['title_tag_count']  =  len(group['tag'].unique())
    print(100*count/len(train_title_group))  

In [ ]:
#求prefix种有多少种title
train_prefix_group = train_data.groupby('prefix')
train_data['prefix_title_count'] = 0
count = 0
for name , group in train_title_group:
    train_data[train_data['prefix'] == name] = len(group['tag'].unique())
    print(100*count/len(train_title_group))

    
print('-'*50)

test_prefix_group = train_data.groupby('prefix')
test_data['prefix_title_count'] = 0
count = 0
for name , group in train_title_group:
    test_data[train_data['prefix'] == name] = len(group['tag'].unique())
    print(100*count/len(train_title_group))


In [6]:
#title点击量
#tag点击量
count = 0
train_data['title_count'] = 0
train_title_group = train_data.groupby(['title'])
for name,group in train_title_group:
    train_data[train_data['title'] == name] = len(group)
    print(100*count/len(train_title_group))


count = 0
test_data['title_count'] = 0
test_title_group = test_data.groupby(['title'])
for name,group in test_title_group:
    test_data[test_data['title'] == name] = len(group)
    print(100*count/len(test_title_group))

train_data['tag_count'] = 0
count = 0
train_tag_group = train_data.groupby(['tag'])
for name,group in train_tag_group:
    train_data[train_data['tag'] == name] = len(group)
    print(100*count/len(train_tag_group))

test_data['tag_count'] = 0
count = 0
test_tag_group = test_data.groupby(['tag'])
for name,group in test_tag_group:
    test_data[test_data['tag'] == name] = len(group)
    print(100*count/len(test_tag_group))

 

        prefix                                   query_prediction  \
1406920     格调  {"格调的拼音": "0.019", "格调娇子多少钱一包": "0.045", "格调是什...   

                        title tag label  title_count  
1406920   LOVA 2010款 1.6AT格调版  汽车     0            0  
--------------------------------------------------
       prefix                                   query_prediction        title  \
377958   forw  {"forword": "0.046", "forward怎么读": "0.025", "f...  ! (Forward)   

       tag label  title_count  
377958  音乐     0            0  
--------------------------------------------------
        prefix                                   query_prediction  \
936969      饭盒  {"饭盒图片大全": "0.187", "饭盒图片大全价格": "0.013", "饭盒袋"...   
1585208     饭盒  {"饭盒图片大全": "0.187", "饭盒图片大全价格": "0.013", "饭盒袋"...   

                 title tag label  title_count  
936969   !我在网上买了个不绣钢饭盒  健康     0            0  
1585208  !我在网上买了个不绣钢饭盒  健康     0            0  
--------------------------------------------------
        prefix     

1338900  商品     0            0  
--------------------------------------------------
        prefix                                   query_prediction      title  \
1615793   宝宝营养  {"宝宝营养不良如何食补": "0.022", "宝宝营养粥做法大全": "0.090", ...  0-6岁宝宝营养餐   
1615804   宝宝营养  {"宝宝营养不良如何食补": "0.022", "宝宝营养粥做法大全": "0.090", ...  0-6岁宝宝营养餐   

        tag label  title_count  
1615793  商品     0            0  
1615804  商品     0            0  
--------------------------------------------------
       prefix                                   query_prediction title tag  \
129298    0.0  {"0.01": "0.017", "0.05%是多少利息": "0.139", "0.05...   0.0  百科   

       label  title_count  
129298     1            0  
--------------------------------------------------
        prefix                                   query_prediction  \
1256925    多少m  {"多少m是一个g": "0.029", "多少m等于1g": "0.060", "多少mb...   

               title tag label  title_count  
1256925  0.25g等于多少mg  健康     0            0  
------------------------------

417704     0            0  
--------------------------------------------------
       prefix                                   query_prediction title tag  \
883452    0号柴  {"0号柴油多少钱一升": "0.026", "0号柴油批发价": "0.031", "0号...  0号柴油  百科   

       label  title_count  
883452     1            0  
--------------------------------------------------
       prefix                                   query_prediction  \
211139     油价  {"油价调整时间": "0.013", "油价最新消息": "0.009", "油价调整最新...   
211143     油价  {"油价调整时间": "0.013", "油价最新消息": "0.009", "油价调整最新...   
211148     油价  {"油价调整时间": "0.013", "油价最新消息": "0.009", "油价调整最新...   
211158     油价  {"油价调整时间": "0.013", "油价最新消息": "0.009", "油价调整最新...   
211159     油价  {"油价调整时间": "0.013", "油价最新消息": "0.009", "油价调整最新...   
211170     油价  {"油价调整时间": "0.013", "油价最新消息": "0.009", "油价调整最新...   
211172     油价  {"油价调整时间": "0.013", "油价最新消息": "0.009", "油价调整最新...   
211178     油价  {"油价调整时间": "0.013", "油价最新消息": "0.009", "油价调整最新...   
211184     油价  {"油价调整时间": "0.013", "油价最新消息": 

1878355  百科     1            0  
--------------------------------------------------
        prefix                                   query_prediction  \
696       1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
25784     1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
40669     1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
127802    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
134714    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
158483    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
249778    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
305015    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
380470    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
448534    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
471760    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
479952    1040  {"1040阳光工程正面报道": "0.058", "1040传销为什么没人查": "0.0...   
541093    1040  {"1

160179  11对战平台为什么现在没有红色警戒的平台了？  知道     0            0  
--------------------------------------------------
         prefix                                query_prediction      title  \
1180246  儿童脚后跟疼  {"儿童脚后跟疼怎么回事": "0.469", "儿童脚后跟疼痛的原因": "0.190"}  11岁儿童脚后跟疼   

        tag label  title_count  
1180246  健康     1            0  
--------------------------------------------------
       prefix                                   query_prediction  \
228467     铜都  {"铜都雕塑": "0.048", "铜都公棚": "0.129", "铜都网": "0.0...   

                                    title tag label  title_count  
228467  11岁孩子喝抗癫痫药十年%2C钙%2C铁%2C锌%2C铜都缺怎么办  健康     0            0  
--------------------------------------------------
       prefix                                   query_prediction  \
601910   鲤鱼打挺  {"鲤鱼打挺怎么练": "0.118", "鲤鱼打挺视频": "0.036", "鲤鱼打挺什...   

                title tag label  title_count  
601910  11岁小孩可以连鲤鱼打挺吗  健康     0            0  
--------------------------------------------------
        prefix 

1847929     1            0  
--------------------------------------------------
       prefix                                   query_prediction    title tag  \
552884    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552887    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552888    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552895    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552922    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552926    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552929    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552934    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552937    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552968    12生  {"12生肖排序": "0.269", "12生肖配对": "0.010", "12生肖图"...  12生肖千相图  百科   
552974    12生  {"12生肖排序": "0.

        prefix                                   query_prediction  title tag  \
1702167   1618  {"1618易米": "0.155", "161810银华内需精选": "0.097", "...  1618年  百科   

        label  title_count  
1702167     0            0  
--------------------------------------------------
        prefix                                   query_prediction title tag  \
1368163   1626  {"162607": "0.092", "162607基金今天净值": "0.124", "...  1626  百科   
1955702   1626  {"162607": "0.092", "162607基金今天净值": "0.124", "...  1626  音乐   

        label  title_count  
1368163     0            0  
1955702     0            0  
--------------------------------------------------
       prefix                                   query_prediction  \
357247    贵州人  {"贵州人口": "0.014", "贵州人事网": "0.021", "贵州人力资源网":...   
357249    贵州人  {"贵州人口": "0.014", "贵州人事网": "0.021", "贵州人力资源网":...   
357255    贵州人  {"贵州人口": "0.014", "贵州人事网": "0.021", "贵州人力资源网":...   
357258    贵州人  {"贵州人口": "0.014", "贵州人事网": "0.021", "贵州人力资源网":...   
357263    贵州人 

KeyboardInterrupt: 